# Module 6 Challenge: Vacation Search

In [1]:
import pandas as pd
import gmaps
import requests

from config import g_key

### Importing CSV derived from WeatherPy_Database

In [2]:
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Talnakh,RU,69.49,88.40,57.20,82,40,17.90,scattered clouds,0.0,0.0
1,1,Prado,BR,-17.34,-39.22,69.93,92,24,3.18,few clouds,0.0,0.0
2,2,Salalah,OM,17.02,54.09,84.20,89,90,4.70,overcast clouds,0.0,0.0
3,3,Clyde River,CA,70.47,-68.59,35.60,93,90,4.70,overcast clouds,0.0,0.0
4,4,Vestmannaeyjar,IS,63.44,-20.27,48.88,83,100,10.67,overcast clouds,0.0,0.0


### Creating a dataframe for minimum and maximum temperature preferences, and if the user wants to see raining or snowing

In [3]:
min_temp = float(input("What is the minimum temperature you would like for your trip?: "))
max_temp = float(input("What is the maximum temperature you would like for your trip?: "))
rainfallYoN = str(input("Do you want it to be raining? (yes/no): "))
snowfallYoN = str(input("Do you want it to be snowing? (yes/no): "))

if rainfallYoN == "yes" and snowfallYoN == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain (inches)"]) & (city_data_df["Snow (inches)"])]
elif rainfallYoN == "yes" and snowfallYoN == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain (inches)"])]
elif rainfallYoN == "no" and snowfallYoN == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Snow (inches)"])]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
    
    
preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip?: 60
What is the maximum temperature you would like for your trip?: 90
Do you want it to be raining? (yes/no): yes
Do you want it to be snowing? (yes/no): no


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
46,46,Bougouni,ML,11.42,-7.48,74.52,81,66,2.21,light rain,0.17,0.0
50,50,Yorosso,ML,12.35,-4.78,79.43,73,72,3.22,light rain,0.59,0.0
93,93,Weyburn,CA,49.67,-103.85,64.99,92,100,5.01,light rain,0.31,0.0
96,96,Iracoubo,GF,5.48,-53.20,78.03,88,100,10.63,moderate rain,1.15,0.0
107,107,Poum,NC,-20.23,164.02,70.50,67,29,18.43,light rain,0.11,0.0
116,116,Maragogi,BR,-9.01,-35.22,75.00,84,2,6.80,light rain,0.18,0.0
124,124,Kitgum,UG,3.28,32.89,65.70,93,79,1.45,moderate rain,1.28,0.0
148,148,Taihe,CN,30.87,105.38,75.20,90,100,5.86,moderate rain,3.23,0.0
156,156,Novikovo,RU,51.15,37.89,62.13,89,23,5.44,moderate rain,1.21,0.0
205,205,Broome,US,42.25,-75.83,66.00,93,90,10.29,light rain,0.76,0.0


### Iterating Through hotel_df DataFrame in order to get hotels near preferred cities

In [4]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [5]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
46,Bougouni,ML,74.52,light rain,11.42,-7.48,Non Stop
50,Yorosso,ML,79.43,light rain,12.35,-4.78,Maison d'arrêt et de correction de Yoro
93,Weyburn,CA,64.99,light rain,49.67,-103.85,Perfect Inns & Suites
96,Iracoubo,GF,78.03,moderate rain,5.48,-53.20,
107,Poum,NC,70.50,light rain,-20.23,164.02,Gîte kajeon
116,Maragogi,BR,75.00,light rain,-9.01,-35.22,Salinas do Maragogi All Inclusive Resort
124,Kitgum,UG,65.70,moderate rain,3.28,32.89,Jaflo Gardens Hotel
148,Taihe,CN,75.20,moderate rain,30.87,105.38,Meifeng Holiday Inn
156,Novikovo,RU,62.13,moderate rain,51.15,37.89,
205,Broome,US,66.00,light rain,42.25,-75.83,Chenango Valley State Park


### Creating HTML info_box and plotting hotel_df into gmaps

In [6]:
gmaps.configure(api_key=g_key)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Saving as CSV file

In [8]:
output_data_file = "weather_data/WeatherPy_vacation.csv"
city_data_df.to_csv(output_data_file, index_label=False)